In [2]:
import pandas as pd
import numpy as np


In [3]:
file = "T2T_SNP_B_MAF_0.05_RMP_Microhap.csv"  # enter the file name

### Naming Scheme MH1,MH2, for all SNPs

In [7]:
df = pd.read_csv(file)
df.head(3)

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,MAF,MajorAF,RMP,Unnamed: 11
0,chrY,5690,rs1433259728,T,A,.,PASS,"COMMON;FREQ=dbGaP_PopFreq:0.5563,0.4437;GNO;RS...",,0.55630,0.378230,
1,chrY,5805,rs1324317743,A,G,.,PASS,"COMMON;FREQ=GnomAD:0.9066,0.09343|dbGaP_PopFre...",0.09004,0.90996,0.712547,
2,chrY,5807,rs1285314756,T,C,.,PASS,"COMMON;FREQ=GnomAD:0.9107,0.08932|dbGaP_PopFre...",0.0897,0.91030,0.713389,NaN


In [24]:
MAX_DISTANCE = 200
SNP_NUM = 3
snp_cluster_list = []
snp_pos_list = df['POS'].tolist()
temp_cluster_list = [snp_pos_list[0]]
temp_group_id_list = []
group_id = 1

# Assign group IDs for all SNPs, default to 'UNGROUPED'
group_ids = ['UNGROUPED'] * len(snp_pos_list)

for i in range(1, len(snp_pos_list)):
    if (snp_pos_list[i] - snp_pos_list[i-1]) <= MAX_DISTANCE:
        temp_cluster_list.append(snp_pos_list[i])
    else:
        if len(temp_cluster_list) >= SNP_NUM:
            snp_cluster_list.extend(temp_cluster_list)
            for idx in range(i - len(temp_cluster_list), i):
                group_ids[idx] = f"MH{group_id}"
            group_id += 1
        temp_cluster_list = [snp_pos_list[i]]

# If at the end of the loop the size of temp_cluster_list is more than 3
if len(temp_cluster_list) >= SNP_NUM:
    snp_cluster_list.extend(temp_cluster_list)
    for idx in range(len(snp_pos_list) - len(temp_cluster_list), len(snp_pos_list)):
        group_ids[idx] = f"MH{group_id}"
    group_id += 1

print(f"snp_pos_list length: {len(snp_pos_list)}")
print(f"group_ids length: {len(group_ids)}")
# print(f"snp_pos_list: {snp_pos_list}")
# print(f"group_ids: {group_ids}")
# Now you can combine as DataFrame:


snp_pos_list length: 11918
group_ids length: 11918


In [32]:
group_series = pd.Series({'GroupID': group_ids})
df["GroupID"] = group_ids
df = df[['CHROM', 'GroupID', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'MAF',
       'MajorAF', 'RMP']]

In [33]:
df.head()

,CHROM,GroupID,POS,ID,REF,ALT,QUAL,FILTER,INFO,MAF,MajorAF,RMP
0,chrY,MH1,5690,rs1433259728,T,A,.,PASS,"COMMON;FREQ=dbGaP_PopFreq:0.5563,0.4437;GNO;RS...",,0.55630,0.378230
1,chrY,MH1,5805,rs1324317743,A,G,.,PASS,"COMMON;FREQ=GnomAD:0.9066,0.09343|dbGaP_PopFre...",0.09004,0.90996,0.712547
2,chrY,MH1,5807,rs1285314756,T,C,.,PASS,"COMMON;FREQ=GnomAD:0.9107,0.08932|dbGaP_PopFre...",0.0897,0.91030,0.713389
3,chrY,MH1,5958,rs1181323098,C,T,.,PASS,"COMMON;FREQ=GnomAD:0.2222,0.7778|dbGaP_PopFreq...",0.3775,0.62250,0.391357
4,chrY,MH1,5993,rs1322758635,T,G,.,PASS,"COMMON;FREQ=dbGaP_PopFreq:0.918,0.08203;GNO;RS...",0.08203,0.91797,0.732817


In [37]:
# save the file
file_save = "output.csv"  # enter the save file name
df.to_csv(file_save, index=False)

GroupID    POS     
MH1        5690        1
           5805        1
           5807        1
           5958        1
           5993        1
                      ..
UNGROUPED  62451230    1
           62452623    1
           62452754    1
           62453642    1
           62453861    1
Name: count, Length: 11918, dtype: int64